# Test evaluation: 
1. Get prediction from CNN/GAN,...
2. Compute baseline
3. Compute evaluation metrics 

In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import xarray as xr
import xskillscore as xs
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
import cartopy.feature as cfeature
import hvplot.xarray
from torch import nn
import cartopy.crs as crs
# add src directory
import sys
sys.path.append('..')
from src.dataloader import *
from src.evaluation import compare_fields, get_eval_mask, get_baseline, compute_eval_metrics

## Get prediction from CNN/GAN...

In [21]:
fn_model = '/datadrive/saved_models/000-test.nc'
test = xr.load_dataset(fn_model)
test.tp.min()
#model = torch.load(fn_model, map_location=torch.device('cpu') )
#model

UpscalingCNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): UpsamplingBilinear2d(scale_factor=2.0, mode=bilinear)
    (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): UpsamplingBilinear2d(scale_factor=2.0, mode=bilinear)
    (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): UpsamplingBilinear2d(scale_factor=2.0, mode=bilinear)
    (1): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): ReLU()
  )
  (conv5): Sequential(
    (0): Conv2d(16, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
)

In [23]:
test.tp

<xarray.DataArray 'tp' (valid_time: 86, lat: 744, lon: 1376)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 2020-05-01 ... 2020-12-01
  * lat         (lat) float64 50.0 49.96 49.92 49.88 ... 20.4 20.36 20.32 20.28
  * lon         (lon) float64 235.0 235.0 235.1 235.1 ... 289.9 290.0 290.0

## Compute baseline 

In [24]:
ds = TiggeMRMSDataset(
tigge_dir='/datadrive/tigge/32km/',
tigge_vars=['total_precipitation'],
mrms_dir='/datadrive/mrms/4km/RadarOnly_QPE_06H/',
rq_fn='/datadrive/mrms/4km/RadarQuality.nc',
val_days=7,
split='valid',
tp_log=0, scale=False,
lead_time=12, # Sofar, this can not yet handle arrays of lead time. 
) 


# 2. Compute baseline 
tigge = ds.tigge.isel(variable=0)
mrms = ds.mrms
baseline = get_baseline(tigge, mrms)


# 3. evaluation mask
eval_mask = get_eval_mask()
eval_mask['lat']=mrms.lat # somehow rq has weird lon lat values! 
eval_mask['lon']=mrms.lon

# 4. compute metrics: 
metrics = compute_eval_metrics(baseline, mrms, eval_mask )
with ProgressBar():
    metrics.load() # execute fss computation    
metrics

NameError: name 'my_f1_score' is not defined